In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
from sklearn.linear_model import LogisticRegression

#### Exercise 12

Implementar regresión Softmax con Batch Gradient Descent y early stopping sin usar sklearn

In [1]:
# Se usa dataset iris
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [6]:
# Descripción del dataset
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [120]:
# Únicamente se van a tener en cuenta los atributos petal length y petal width por su alta correlación
# con la clase
X = iris['data'][:, (2, 3)]
y = iris['target']

In [122]:
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10, random_state=42)
softmax_reg.fit(X, y)

LogisticRegression(C=10, multi_class='multinomial', random_state=42)

In [125]:
# Se añade el término bias
X = np.c_[np.ones([len(X), 1]), X]

In [127]:
# Se añade semilla para reproducir pasos
np.random.seed(2042)

In [149]:
# Se hace la división manual del dataset en train, validation y test
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - validation_size - test_size

indices_shuffled = np.random.permutation(total_size)

X_train = X[indices_shuffled[:train_size]]
y_train = y[indices_shuffled[:train_size]]

X_valid = X[indices_shuffled[train_size:train_size + validation_size]]
y_valid = y[indices_shuffled[train_size:train_size + validation_size]]

X_test = X[indices_shuffled[-test_size:]]
y_test = y[indices_shuffled[-test_size:]]

# Se chequea la suma de instancias de los 3 datasets
print('Suma X: ', len(X_train) + len(X_valid) + len(X_test), ", longitud dataset original (X): ", len(X))
print('Suma y: ', len(y_train) + len(y_valid) + len(y_test), ", longitud dataset original (y): ", len(y))

Suma X:  150 , longitud dataset original (X):  150
Suma y:  150 , longitud dataset original (y):  150


In [156]:
# Se transforma el vector de etiquetas con 0, 1 y 2 en una matriz donde cada instancia se representa
# por un vector de codificación one-hot
def target_to_one_hot(y):
    y_one_hot = np.zeros((len(y), y.max() + 1))
    y_one_hot[np.arange(len(y)), y] = 1
    return y_one_hot

In [157]:
# Chequeo de la transformación de la target
y_train[:5], target_to_one_hot(y_train[:5]) 

(array([1, 0, 0, 2, 1]),
 array([[0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.]]))

In [158]:
y_train_one_hot = target_to_one_hot(y_train)
y_valid_one_hot = target_to_one_hot(y_valid)
y_test_one_hot = target_to_one_hot(y_test)

In [175]:
# Se crea el vector de salida de la regresión logística en una distribución de probabilidades (softmax)
def softmax(logits):
    exps = np.exp(logits)
    return exps / np.sum(exps, axis=1, keepdims=True)

In [239]:
# Sin regularización
n_iterations = 5001
eta = 0.01
epsilon = 1e-7
m = len(X_train)

Theta = np.random.randn(X_train.shape[1], len(np.unique(y_train)))

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    y_proba = softmax(logits)
    if iteration % 500 ==0:
        loss = -np.mean(np.sum(y_train_one_hot*np.log(y_proba + epsilon) + 
                               (1-y_train_one_hot)*np.log(1-y_proba + epsilon), axis=1))
        print(iteration, loss)
    gradients = 1/m * X_train.T.dot(y_proba - y_train_one_hot)
    Theta = Theta - eta * gradients


0 8.138020401231916
500 1.2380424287718161
1000 1.1103607913148847
1500 1.0201750732117336
2000 0.9524356126796059
2500 0.8989987533267135
3000 0.8552284076996501
3500 0.8183295571115614
4000 0.7865248917607133
4500 0.7586301760620614
5000 0.7338239662414254


In [240]:
Theta

array([[ 3.30019937e+00, -4.78656842e-01, -2.55387129e+00],
       [-4.72043115e-01,  5.99182104e-01, -6.30460812e-02],
       [-1.50361040e+00, -1.25584453e-03,  3.21765630e+00]])

In [241]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9

In [249]:
# Con regularización l2
n_iterations = 5001
eta = 0.01
epsilon = 1e-7
m = len(X_train)
alpha = 0.1

Theta = np.random.randn(X_train.shape[1], len(np.unique(y_train)))

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    y_proba = softmax(logits)
    if iteration % 500 ==0:
        entropy_loss = -np.mean(np.sum(y_train_one_hot*np.log(y_proba + epsilon) + 
                               (1-y_train_one_hot)*np.log(1-y_proba + epsilon), axis=1))
        
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
        loss = entropy_loss + alpha * l2_loss
        print(iteration, loss)
    gradients = 1/m * X_train.T.dot(y_proba - y_train_one_hot) + np.r_[np.zeros([1, len(np.unique(y_train))]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

0 3.6720804075378197
500 1.4389974843895008
1000 1.2637826352567512
1500 1.1664046158936359
2000 1.1061929731710547
2500 1.0654039420351227
3000 1.0357514767297176
3500 1.0130275641683326
4000 0.9949138047025028
4500 0.98003837014138
5000 0.9675395588482029


In [250]:
# A pesar de que el loss obtenido con regularización es mas grande que el obtenido
# sin regularización, el modelo obtiene un mejor rendimiento. Aumenta un 6% la
# accuracy
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667

#### Early stopping

In [262]:
# Con regularización l2
n_iterations = 5001
eta = 0.01
epsilon = 1e-7
m = len(X_train)
alpha = 0.1
best_loss = np.infty

Theta = np.random.randn(X_train.shape[1], len(np.unique(y_train)))

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    y_proba = softmax(logits)
    entropy_loss = -np.mean(np.sum(y_train_one_hot*np.log(y_proba + epsilon) + 
                               (1-y_train_one_hot)*np.log(1-y_proba + epsilon), axis=1))
        
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = entropy_loss + alpha * l2_loss
    
    gradients = 1/m * X_train.T.dot(y_proba - y_train_one_hot) + np.r_[np.zeros([1, len(np.unique(y_train))]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients
    
    if iteration % 500 ==0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration -1, best_loss)
        print(iteration, loss, "early stopping!")
        break

0 5.740951548243137
500 1.2456017287048153
1000 1.1366375994396596
1500 1.074025646201764
2000 1.0342943119823265
2500 1.0068580147255584
3000 0.9866086777951901
3500 0.970891576106112
4000 0.958222228910103
3999 0.958222228910103
4000 0.958222228910103 early stopping!


In [263]:
# Misma accuracy pero en menos tiempo
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667